In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.environ['KAGGLE_USERNAME'] = "lampe01" # username from the json file
os.environ['KAGGLE_KEY'] = "0fe21bb583c2cd3230e9d928c96e27f7" # key from the json file
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/CirCor"

In [3]:
%cd /content/drive/MyDrive/CirCor

[Errno 2] No such file or directory: '/content/drive/MyDrive/CirCor'
/content


In [4]:
pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
pip install ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
pip install Ipython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
pip uninstall scikit-learn

Found existing installation: scikit-learn 0.24.2
Uninstalling scikit-learn-0.24.2:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/scikit_learn-0.24.2.dist-info/*
    /usr/local/lib/python3.7/dist-packages/scikit_learn.libs/libgomp-f7e03b3e.so.1.0.0
    /usr/local/lib/python3.7/dist-packages/sklearn/*
Proceed (y/n)? y
  Successfully uninstalled scikit-learn-0.24.2


In [8]:
pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scikit_learn-1.0.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (24.8 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyaudioprocessing 1.1.13 requires scikit-learn==0.24.2; python_version <= "3.7", but you have scikit-learn 1.0.2 which is incompatible.


In [9]:
pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
pip install pyAudioProcessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scikit_learn-0.24.2-cp37-cp37m-manylinux2010_x86_64.whl (22.3 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.5 requires scikit-learn>=1.0.0, but you have scikit-learn 0.24.2 which is incompatible.


In [11]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
import itertools
import os
import sys
import pywt
import librosa
import sklearn
import scipy
import scipy.io
import scipy.io.wavfile
from scipy import signal
from scipy.stats import kurtosis, skew, tstd
from scipy.signal import periodogram, welch, butter, lfilter, freqz
from os import path
from pydub import AudioSegment
import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras.models import Model, load_model, Sequential
from keras.layers import Input, Dense, Dropout, AlphaDropout, Activation, Flatten, LSTM
from keras.callbacks import ModelCheckpoint, TensorBoard,EarlyStopping,ReduceLROnPlateau,ProgbarLogger
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, precision_recall_curve, recall_score, classification_report, auc, roc_curve, precision_recall_fscore_support, f1_score, accuracy_score
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from pyAudioProcessing.extract_features import get_features

In [12]:
mdata = pd.read_csv('/content/drive/MyDrive/ReCirCor/training_data.csv')
input_dir = '/content/drive/MyDrive/ReCirCor/training_data/training_data'

In [13]:
#Gets all the txt files
def getfilenames(inputdir):
    Fname = list()
    Fname = sorted([fname for fname in os.listdir(inputdir) if not fname.startswith('.')and fname.endswith('.txt')])
    
    return Fname

In [14]:
#List of Labels - 0 for normal, 1 for murmur
labels = [0,0,0,0,0,1,1,1,1,1,1,1,1,0,1,1,1,0,0,0,1,1,1]
len(labels)

23

In [15]:
#They were originally named differently and I have renamed and numbered them for ease of access.
#The michigan heart sound recordings are in mp3 format
#The recordings are converted to wav format for easy processing as most python libraries require the audio format to be wav.

def converttowav(filepath):
  destinationpath = filepath.split('.')[0] + '.wav'
  sound = AudioSegment.from_mp3(filepath)
  sound.export(destinationpath, format = "wav")

filepathslist = list()
for i in range(23):
  filepath = str(i+1) + '.mp3'
  converttowav(filepath)
  filepathslist.append(str(i+1)+'.wav')

In [16]:
#Creating an empty dataframe with columns to store filenames and the respective labels
dataframe = pd.DataFrame(columns = ['filename', 'labels'])

In [17]:
filepathslist

['1.wav',
 '2.wav',
 '3.wav',
 '4.wav',
 '5.wav',
 '6.wav',
 '7.wav',
 '8.wav',
 '9.wav',
 '10.wav',
 '11.wav',
 '12.wav',
 '13.wav',
 '14.wav',
 '15.wav',
 '16.wav',
 '17.wav',
 '18.wav',
 '19.wav',
 '20.wav',
 '21.wav',
 '22.wav',
 '23.wav']

In [18]:
labels

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1]

In [19]:
labels[18]

0

In [20]:
#Inserting filename and label values into dataframe
dataframe['filename'] = filepathslist
dataframe['labels'] = labels

In [21]:
dataframenew = pd.DataFrame(columns = ['new ID', 'wav filename', 'labels'])

In [22]:
import random

In [23]:
newID = list()
wavfilename = list()
newlabels = list()
for i in range(1000):
  x = random.randint(1,23)
  newID.append(str(i+1))
  wavfilename.append(str(x)+'.wav')
  newlabels.append(labels[x-1])

In [24]:
#Get only murmur recordings
def getrecmur(inputdir):
    fname = getfilenames(inputdir)
    items = []
    rinfo = []
    recf = []
    recfile = []
    for i in fname:
        i = inputdir + '/' + i
        with open(i, 'r') as f:
            content = f.readlines()
        numloc = int(content[0].split(' ')[1])
        for i in range(numloc):
            for p in range(len(content)):
                t = content[p].split('\n')
                for it in t:
                    if it.startswith('#Murmur:'):
                        tar = it.split(': ')[1]
                        if tar == 'Present':
                          recinfo = content[i+1].split(' ')
                          rinfo.append(recinfo)
    for items in rinfo:
        for i in range(len(items)):
            if items[i].endswith('.wav'):
                recfile = input_dir + '/' + items[i]
        recf.append(recfile)
    return rinfo, recf

rin, ref = getrecmur(input_dir)

In [25]:
len(rin)

616

In [26]:
ref

['/content/drive/MyDrive/ReCirCor/training_data/training_data/13918_AV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/13918_PV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/13918_TV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/13918_MV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/14241_AV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/14241_PV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/14241_TV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/14241_MV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/29045_AV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/29045_PV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/29045_TV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/29045_MV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/33151_AV.wav',

In [27]:
x = 1000
for i in ref:
  x = x + 1
  l = 1
  wavfilename.append(i)
  newlabels.append(l)
  newID.append(x)

In [28]:
len(newID)

1616

In [29]:
dataframenew['new ID'] = newID
dataframenew['wav filename'] = wavfilename
dataframenew['labels'] = newlabels
dataframenew

,new ID,wav filename,labels
0,1,23.wav,1
1,2,10.wav,1
2,3,1.wav,0
3,4,14.wav,0
4,5,5.wav,0
...,...,...,...
1611,1612,/content/drive/MyDrive/ReCirCor/training_data/...,1
1612,1613,/content/drive/MyDrive/ReCirCor/training_data/...,1
1613,1614,/content/drive/MyDrive/ReCirCor/training_data/...,1
1614,1615,/content/drive/MyDrive/ReCirCor/training_data/...,1


In [30]:
dataframe

,filename,labels
0,1.wav,0
1,2.wav,0
2,3.wav,0
3,4.wav,0
4,5.wav,0
5,6.wav,1
6,7.wav,1
7,8.wav,1
8,9.wav,1
9,10.wav,1


In [31]:
dataframenew['labels'].value_counts()

1    1216
0     400
Name: labels, dtype: int64

In [32]:
X_train, X_test, y_Train, y_Test = train_test_split(dataframenew['new ID'], dataframenew['labels'], train_size=0.8, shuffle=True)

In [33]:
X_train

1026    1027
1364    1365
606      607
640      641
179      180
        ... 
600      601
418      419
1100    1101
122      123
1579    1580
Name: new ID, Length: 1292, dtype: object

In [34]:
X_test

264      265
1367    1368
1357    1358
1283    1284
1294    1295
        ... 
126      127
662      663
223      224
637      638
920      921
Name: new ID, Length: 324, dtype: object

In [35]:
#Computing Features - 

#Discrete Wavelet Transform
def computeDWTs(x):
    #approximation and detail coefficients
    res = np.array([])
    ca, cd = pywt.dwt(x, 'db8')
    #Compute mean, stdev, skew, kurtosis
    camean = np.mean(ca)
    res = np.hstack((res,camean))
    castdev = tstd(ca)
    res = np.hstack((res,castdev))
    caskew = skew(ca)
    res = np.hstack((res,caskew))
    cakurtosis = kurtosis(ca)
    res = np.hstack((res,cakurtosis))
    cdmean = np.mean(cd)
    res = np.hstack((res,cdmean))
    cdstdev = tstd(cd)
    res = np.hstack((res,cdstdev))
    cdskew = skew(cd)
    res = np.hstack((res,cdskew))
    cdkurtosis = kurtosis(cd)
    res = np.hstack((res, cdkurtosis))
    return res

#Computing the Spectral Entropy of the signal - Shannon Entropy of the Power Spectral Density (PSD) of the data
def SpectralEntropy(Signal, SampleRate, method = 'fft', nperseg = None, normalize = False):
    Signal = np.array(Signal)
    if method == 'fft':
      _, psd = periodogram(Signal, SampleRate)
    elif method == 'welch':
      _, psd = welch(Signal, SampleRate, nperseg=nperseg)
    psd_norm = np.divide(psd, psd.sum())
    se = -np.multiply(psd_norm, np.log2(psd_norm)).sum()
    if normalize:
        se /= np.log2(psd_norm.size)
    return se

#Computing the Power Spectral Density(PSD) of signal - 
def computepsd(x):
    freq, psd = signal.welch(x)
    return psd

#Computing Mel-frequency cepstral coefficients, Gammatone frequency cepstral coefficients, RMS Energy, Zero-crossing rate, mean, variance, skew, and kurtosis of the signal
def computeMFCC(x, sr):
    mfccs = np.mean(librosa.feature.mfcc(y = x, sr = sr, n_mfcc = 13).T, axis = 0)
    return mfccs

def computeGFCC(file_name):
    gfcc = get_features(file=file_name, feature_names=["gfcc"])
    gfcc = np.array(gfcc['audio'][file_name]['features'])
    return gfcc

def computeRMSEn(x):
    rmsen = np.mean(librosa.feature.rms(x, frame_length=2048, hop_length=1024, center=True).T, axis = 0)
    return rmsen

def computeZCR(x):
    zcr = np.mean(librosa.feature.zero_crossing_rate(y = x, frame_length=2048, hop_length=1024).T, axis = 0)
    return zcr

def computestats(x):
#Mean, Variance, Skew, Kurtosis
    stats = []
    me = np.mean(x)
    stats.append(me)
    variance = np.var(x)
    stats.append(variance)
    sk = skew(x)
    stats.append(sk)
    kurt = kurtosis(x)
    stats.append(kurt)
    return stats

In [36]:
#Function to extract the required features
def extractfeatures(file_name, x, sr, *, mfcc: bool = False, dwt: bool = False, spectral_entropy: bool = False, psd: bool = False, gfcc: bool = False, rmse: bool = False, zcr: bool = False, stats: bool = False):
    finalfeatures = np.array([])
    if mfcc:
        mfccs = computeMFCC(x, sr)
        finalfeatures = np.hstack((finalfeatures, mfccs))
    if dwt:
        dwtr = computeDWTs(x)
        finalfeatures = np.hstack((finalfeatures, dwtr))
    if spectral_entropy:
        specen = SpectralEntropy(x, sr)
        finalfeatures = np.hstack((finalfeatures,specen))
    if psd:
        psds = computepsd(x)
        finalfeatures = np.hstack((finalfeatures, psds))
    if gfcc:
        gfccs = computeGFCC(file_name)
        finalfeatures = np.hstack((finalfeatures, gfccs))
    if rmse:
        rmses = computeRMSEn(x)
        finalfeatures = np.hstack((finalfeatures, rmses))
    if zcr:
        zcrs = computeZCR(x)
        finalfeatures = np.hstack((finalfeatures, zcrs))
    if stats:
        stats1 = computestats(x)
        finalfeatures = np.hstack((finalfeatures, stats1))
    return finalfeatures

In [37]:
#loads audio and extracts required features
def load_file_data(file_num, duration=15, sr = 4000):
    input_length=sr*duration
    notfeatures = []
    i = 0
    for file_no in file_num:
        try:
            #sound_file = folder + '/' + file_name
            sfile = int(file_no)
            sound_file = dataframenew['wav filename'][sfile-1]
            print ("load file ", i, sound_file)
            x, sr = librosa.load(sound_file,duration=duration,res_type='kaiser_fast')
            dur = librosa.get_duration(y=x, sr=sr)
            #Padding audio signal
            if (round(dur) < duration):
                print ("fixing audio length:", file_no, sound_file)
                y = librosa.util.fix_length(x, input_length)
            feature = extractfeatures(sound_file, x, sr, mfcc = True, dwt = True, psd = True, rmse = True, zcr = True,stats = True)
            notfeatures.append(feature)          
        except Exception as e:
            print(e)
            #notfeatures = np.asarray(features, dtype=np.float32)
        i = i+1 
    return notfeatures

In [38]:
x_train = []
x_test = []
x_train = load_file_data(X_train)
x_test = load_file_data(X_test)

load file  0 /content/drive/MyDrive/ReCirCor/training_data/training_data/44514_TV.wav
load file  1 /content/drive/MyDrive/ReCirCor/training_data/training_data/69068_MV.wav
load file  2 20.wav
load file  3 14.wav
load file  4 22.wav
load file  5 11.wav
load file  6 /content/drive/MyDrive/ReCirCor/training_data/training_data/83094_MV.wav
load file  7 3.wav
load file  8 2.wav
load file  9 16.wav
load file  10 16.wav
load file  11 /content/drive/MyDrive/ReCirCor/training_data/training_data/69096_TV.wav
fixing audio length: 1376 /content/drive/MyDrive/ReCirCor/training_data/training_data/69096_TV.wav
load file  12 23.wav
load file  13 /content/drive/MyDrive/ReCirCor/training_data/training_data/49712_TV.wav
load file  14 7.wav
load file  15 13.wav
load file  16 5.wav
load file  17 /content/drive/MyDrive/ReCirCor/training_data/training_data/50115_AV.wav
load file  18 /content/drive/MyDrive/ReCirCor/training_data/training_data/68306_MV.wav
load file  19 /content/drive/MyDrive/ReCirCor/training

In [39]:
x_train = np.asarray(x_train).astype(np.float32)
x_test = np.asarray(x_test).astype(np.float32)

In [40]:
x_test.shape

(324, 156)

In [41]:
x_train.shape

(1292, 156)

In [42]:
y_Train.dtype

dtype('int64')

In [43]:
y_Train = np.asarray(y_Train).astype(np.float32)

In [44]:
y_Test = np.asarray(y_Test).astype(np.float32)

In [45]:
traindataframe = pd.DataFrame(columns = ['x_train ID', 'labels'])
testdataframe = pd.DataFrame(columns = ['x_test ID', 'labels'])
traindataframe['x_train ID'] = X_train
testdataframe['x_test ID'] = X_test
traindataframe['labels'] = y_Train
testdataframe['labels'] = y_Test

In [46]:
testdataframe

,x_test ID,labels
264,265,0.0
1367,1368,1.0
1357,1358,1.0
1283,1284,1.0
1294,1295,1.0
...,...,...
126,127,1.0
662,663,1.0
223,224,1.0
637,638,1.0


In [47]:
xtrainwavfiles = list()
for i in X_test:
  num = int(i)
  xtrainwavfiles.append(dataframenew['wav filename'][num-1])

In [48]:
pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [49]:
import kerastuner
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization
def model_builder(hp):
  model = Sequential()
  hp_units1 = hp.Int('units_', min_value=2, max_value=64, step=4)
  model.add(tf.keras.layers.Dense(units = hp_units1, input_dim = x_train.shape[1]))

  # Tune the number of dense layers
  for i in range(hp.Int('num_layers', 1, 5)):
    
    # Tune the number of units in the each dense layer
    hp_units = hp.Int('units_'+str(i), min_value=2, max_value=64, step=4)
    model.add(tf.keras.layers.Dense(units=hp_units, activation='relu'))
    
    # Tune the dropout rate in the each dense layer
    hp_dropout = hp.Float('rate', min_value=0.0, max_value=0.5, step=0.1)
    model.add(tf.keras.layers.Dropout(hp_dropout))
    
  # Add dense output layer
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

  # Tune the learning rate for the optimizer
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),loss='binary_crossentropy',
                metrics=['accuracy'])

  return model

#tf.keras.metrics.Recall()
#tuner = BayesianOptimization(model_builder,
#max_trials=50,
#objective='val_accuracy',
#overwrite=True)

tuner = Hyperband(model_builder,
objective=kerastuner.Objective("val_accuracy", direction="max"),
max_epochs=50,
factor=3, overwrite = True)

tuner.search(x_train,y_Train, epochs=50, validation_split=0.2)
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

model = tuner.hypermodel.build(best_hps)

Trial 90 Complete [00h 00m 09s]
val_accuracy: 0.8339768052101135

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 05m 00s


In [50]:
history = model.fit(x_train, y_Train, batch_size = 10, epochs=50, validation_split = 0.2, verbose = 1)

Epoch 1/50
104/104 [==============================] - 2s 9ms/step - loss: 99.5350 - accuracy: 0.7957 - val_loss: 22.9523 - val_accuracy: 0.8571
Epoch 2/50
104/104 [==============================] - 1s 8ms/step - loss: 29.9439 - accuracy: 0.8906 - val_loss: 0.5854 - val_accuracy: 0.9189
Epoch 3/50
104/104 [==============================] - 1s 5ms/step - loss: 17.9028 - accuracy: 0.9177 - val_loss: 4.6054 - val_accuracy: 0.9189
Epoch 4/50
104/104 [==============================] - 0s 4ms/step - loss: 6.8991 - accuracy: 0.9371 - val_loss: 1.2184 - val_accuracy: 0.9382
Epoch 5/50
104/104 [==============================] - 0s 4ms/step - loss: 2.8183 - accuracy: 0.9303 - val_loss: 3.1110 - val_accuracy: 0.9189
Epoch 6/50
104/104 [==============================] - 0s 4ms/step - loss: 18.5276 - accuracy: 0.9148 - val_loss: 1.1959 - val_accuracy: 0.9691
Epoch 7/50
104/104 [==============================] - 1s 7ms/step - loss: 3.5139 - accuracy: 0.9439 - val_loss: 1.1042 - val_accuracy: 0.9498
E

In [51]:
y_predict = np.argmax(model.predict(x_test), axis=-1)
y_pred = model.predict(x_test)
y_pred=np.argmax(y_pred, axis=1)

11/11 [==============================] - 0s 4ms/step


In [52]:
print("Classification Report : \n\n", classification_report(y_Test, y_pred))

Classification Report : 

               precision    recall  f1-score   support

         0.0       0.23      1.00      0.38        76
         1.0       0.00      0.00      0.00       248

    accuracy                           0.23       324
   macro avg       0.12      0.50      0.19       324
weighted avg       0.06      0.23      0.09       324



In [53]:
df = pd.DataFrame({'Input':X_test, 'Wav filename':xtrainwavfiles, 'Real Values':y_Test, 'Predicted Values':y_pred})
df

,Input,Wav filename,Real Values,Predicted Values
264,265,1.wav,0.0,0
1367,1368,/content/drive/MyDrive/ReCirCor/training_data/...,1.0,0
1357,1358,/content/drive/MyDrive/ReCirCor/training_data/...,1.0,0
1283,1284,/content/drive/MyDrive/ReCirCor/training_data/...,1.0,0
1294,1295,/content/drive/MyDrive/ReCirCor/training_data/...,1.0,0
...,...,...,...,...
126,127,6.wav,1.0,0
662,663,13.wav,1.0,0
223,224,21.wav,1.0,0
637,638,6.wav,1.0,0


In [54]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
clf = AdaBoostClassifier(n_estimators=100, random_state=0)
clf.fit(x_train, y_Train)

AdaBoostClassifier(n_estimators=100, random_state=0)

In [55]:
clf.score(x_test, y_Test)

1.0

In [56]:
y_pred1 = clf.predict(x_test)
print("Classification Report : \n\n", classification_report(y_Test, y_pred1))

Classification Report : 

               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        76
         1.0       1.00      1.00      1.00       248

    accuracy                           1.00       324
   macro avg       1.00      1.00      1.00       324
weighted avg       1.00      1.00      1.00       324



In [57]:
df1 = pd.DataFrame({'Input':X_test, 'Wav filename':xtrainwavfiles, 'Real Values':y_Test, 'Predicted Values':y_pred1})
df1

,Input,Wav filename,Real Values,Predicted Values
264,265,1.wav,0.0,0.0
1367,1368,/content/drive/MyDrive/ReCirCor/training_data/...,1.0,1.0
1357,1358,/content/drive/MyDrive/ReCirCor/training_data/...,1.0,1.0
1283,1284,/content/drive/MyDrive/ReCirCor/training_data/...,1.0,1.0
1294,1295,/content/drive/MyDrive/ReCirCor/training_data/...,1.0,1.0
...,...,...,...,...
126,127,6.wav,1.0,1.0
662,663,13.wav,1.0,1.0
223,224,21.wav,1.0,1.0
637,638,6.wav,1.0,1.0


In [58]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

In [59]:
clf2 = LinearSVC(penalty='l2', loss='hinge')
clf2.fit(x_train, y_Train)
clf2.score(x_test, y_Test)

0.9290123456790124

In [60]:
y_pred2 = clf2.predict(x_test)
print("Classification Report : \n\n", classification_report(y_Test, y_pred2))

Classification Report : 

               precision    recall  f1-score   support

         0.0       0.77      1.00      0.87        76
         1.0       1.00      0.91      0.95       248

    accuracy                           0.93       324
   macro avg       0.88      0.95      0.91       324
weighted avg       0.95      0.93      0.93       324



In [61]:
df2 = pd.DataFrame({'Input':X_test, 'Wav filename':xtrainwavfiles, 'Real Values':y_Test, 'Predicted Values':y_pred2})
df2

,Input,Wav filename,Real Values,Predicted Values
264,265,1.wav,0.0,0.0
1367,1368,/content/drive/MyDrive/ReCirCor/training_data/...,1.0,1.0
1357,1358,/content/drive/MyDrive/ReCirCor/training_data/...,1.0,1.0
1283,1284,/content/drive/MyDrive/ReCirCor/training_data/...,1.0,1.0
1294,1295,/content/drive/MyDrive/ReCirCor/training_data/...,1.0,1.0
...,...,...,...,...
126,127,6.wav,1.0,1.0
662,663,13.wav,1.0,1.0
223,224,21.wav,1.0,1.0
637,638,6.wav,1.0,1.0


In [62]:
clf4 = SGDClassifier(loss = 'hinge').fit(x_train, y_Train)

In [63]:
clf4.score(x_test, y_Test)

1.0

In [64]:
y_pred4 = clf4.predict(x_test)
#y_pred=np.argmax(y_pred, axis=1)
print("Classification Report : \n\n", classification_report(y_Test, y_pred4))

Classification Report : 

               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        76
         1.0       1.00      1.00      1.00       248

    accuracy                           1.00       324
   macro avg       1.00      1.00      1.00       324
weighted avg       1.00      1.00      1.00       324



In [65]:
df4 = pd.DataFrame({'Input':X_test, 'Wav filename':xtrainwavfiles, 'Real Values':y_Test, 'Predicted Values':y_pred4})
df4

,Input,Wav filename,Real Values,Predicted Values
264,265,1.wav,0.0,0.0
1367,1368,/content/drive/MyDrive/ReCirCor/training_data/...,1.0,1.0
1357,1358,/content/drive/MyDrive/ReCirCor/training_data/...,1.0,1.0
1283,1284,/content/drive/MyDrive/ReCirCor/training_data/...,1.0,1.0
1294,1295,/content/drive/MyDrive/ReCirCor/training_data/...,1.0,1.0
...,...,...,...,...
126,127,6.wav,1.0,1.0
662,663,13.wav,1.0,1.0
223,224,21.wav,1.0,1.0
637,638,6.wav,1.0,1.0
